In [ ]:
import argparse
import pandas as pd
from sklearn import svm

**Step 1: read in input args**

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--train_path', type=str, required=True)    # path to training data
# parser.add_argument('--test_path', type=str, required=True)     # path to test data 
# parser.add_argument('--output_path', type=str, required=True)   # path to output file (metric results)
# args = parser.parse_args()

TRAIN_PATH = "/Users/lilykawaoto/Documents/GitHub/LING-L715/lhsab_train.tsv"
TEST_PATH = "/Users/lilykawaoto/Documents/GitHub/LING-L715/lhsab_test.tsv"
OUTPUT_PATH = "/Users/lilykawaoto/Documents/GitHub/LING-L715/lhsab_svm_output.txt"

# with open(args.train_path, 'r') as train_f:
with open(TRAIN_PATH, 'r') as train_f:
    col_names = ["text", "label"]
    train_df = pd.read_csv(train_f, delimiter="\t", names=col_names)
    # df.head()
    x_train = [row[0] for row in train_df.itertuples(index=False)]
    y_train = [row[1] for row in train_df.itertuples(index=False)]

# with open(args.test_path, 'r') as test_f:
with open(TEST_PATH, 'r') as test_f:
    col_names = ["text", "label"]
    test_df = pd.read_csv(train_f, delimiter="\t", names=col_names)
    x_test = [row[0] for row in test_df.itertuples(index=False)]
    y_test = [row[1] for row in test_df.itertuples(index=False)]

**Step 2:   create unigrams, unigrams+bigrams, unigrams+bigrams+trigrams
            create term-frequency ratings with thresholds of 2 & 3   // term-freq: the relative freq of a term t within a document d**

In [ ]:
def make_ngram(n, threshold, txt_list):
    ngrams_dict = {}
    # ngrams_list = []
    for txt in txt_list:
        if n==1:        # make unigrams
            words = txt.split(" ")
            for word in words:
                # ngrams_list.append(word)
                if ngrams_dict[word] == 0:
                    ngrams_dict[word] = 1
                else:
                    ngrams_dict[word] += 1

        elif n==2:      # make bigrams
            words = txt.split(" ")
            for i,word in enumerate(words):
                if i==0:
                    continue
                else:
                    # ngrams_list.append((txt[i-1], word))
                    if ngrams_dict[(words[i-1], word)] == 0:
                        ngrams_dict[(words[i-1], word)] = 1
                    else:
                        ngrams_dict[(words[i-1], word)] += 1

        elif n==3:      # make trigrams
            words = txt.split(" ")
            if len(words) < 3:
                continue
            for i,word in enumerate(words):
                if i<2:
                    continue
                else: 
                    # ngrams_list.append((txt[i-2], txt[i-1], word))
                    if ngrams_dict[(words[i-2], words[i-1], word)] == 0:
                        ngrams_dict[(words[i-2], words[i-1], word)] = 1
                    else: 
                        ngrams_dict[(words[i-2], words[i-1], word)] += 1

    assert(threshold==2 or threshold==3)
    if threshold==2:
        ngrams_dict = {key:val for key, val in ngrams_dict.items() if val>2}

    elif threshold==3:
        ngrams_dict = {key:val for key, val in ngrams_dict.items() if val>3}

    return ngrams_dict

In [ ]:
# threshold == 2
train_unigrams_2 = list(make_ngram(1, 2, x_train).keys())
train_bigrams_2 = list(make_ngram(2, 2, x_train).keys())
train_trigrams_2 = list(make_ngram(3, 2, x_train).keys())

test_unigrams_2 = list(make_ngram(1, 2, x_train).keys())
test_bigrams_2 = list(make_ngram(2, 2, x_train).keys())
test_trigrams_2 = list(make_ngram(3, 2, x_train).keys())

In [ ]:
# threshold == 3
train_unigrams_3 = list(make_ngram(1, 3, x_train).keys())
train_bigrams_3 = list(make_ngram(2, 3, x_train).keys())
train_trigrams_3 = list(make_ngram(3, 3, x_train).keys())

test_unigrams_3 = list(make_ngram(1, 3, x_train).keys())
test_bigrams_3 = list(make_ngram(2, 3, x_train).keys())
test_trigrams_3 = list(make_ngram(3, 3, x_train).keys())

**Step 3: train & predict**

In [ ]:
# unigrams, threshold 2
clf = svm.SVC(kernel='linear')
clf.fit(train_unigrams_2, y_train)
y_pred = clf.predict(test_unigrams_2)

In [ ]:
# unigrams + bigrams, threshold 2
clf = svm.SVC(kernel='linear')
clf.fit(train_unigrams_2+train_bigrams_2, test_unigrams_2+test_bigrams_2)
y_pred = clf.predict(test_unigrams_2+test)